In [12]:
# Painful crude verification that 1D winograd from `wincnn.py` matches with `np.convolve`
import wincnn
import numpy as np
import sympy as sp
import time

In [13]:
# n is input size
# r is filter size
n = 2
r = 3
alpha = n + r - 1

# Winograd
AT, G, BT, f = wincnn.cookToomFilter((0, 1, -1), 2, 3, wincnn.FractionsInG)

B = BT.transpose()
A = AT.transpose()

di = wincnn.IndexedBase('d')
gi = wincnn.IndexedBase('g')

d = wincnn.Matrix(n, 1, lambda i, j: di[i])
g = wincnn.Matrix(r, 1, lambda i, j: gi[i])

V = A * d
U = G * g

M = U.multiply_elementwise(V)
Y = wincnn.simplify(B * M)

d0, d1, g0, g1, g2 = sp.symbols('d0 d1 g0 g1 g2')

Y_indexed = sp.Matrix([
    [            d0*g0],
    [d0*g1 + d1*g0],
    [d0*g2 + d1*g1],
    [            d1*g2]])

/home/yppatel/mcufno/wincnn.py:19: SymPyDeprecationWarning: 

non-Expr objects in a Matrix is deprecated. Matrix represents
a mathematical matrix. To represent a container of non-numeric
entities, Use a list of lists, TableForm, NumPy array, or some
other data structure instead.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-non-expr-in-matrix
for details.

This has been deprecated since SymPy version 1.9. It
will be removed in a future version of SymPy.

  return Matrix(n, 1, lambda i,j: Poly((reduce(mul, ((x-a[k] if k!=i else 1) for k in range(0,n)), 1)).expand(basic=True), x))


In [18]:
trials   = 10_000
d_values_trials = np.random.random((trials, n)) 
g_values_trials = np.random.random((trials, r))

In [19]:
def time_method(method_name, d_values_trials, g_values_trials, trials=10_000):
    t0 = time.time()
    for trial in range(trials):
        d_values, g_values = d_values_trials[trial], g_values_trials[trial]
        if method_name == "direct":
            result = np.convolve(d_values, g_values)
        elif method_name == "wino":
            subs = {d0: d_values[0], d1: d_values[1], g0: g_values[0], g1: g_values[1], g2: g_values[2]}
            result = Y_indexed.subs(subs)
    t1 = time.time()
    return t1-t0, result

In [20]:
time_direct = time_method("direct", d_values_trials, g_values_trials)
time_wino   = time_method("wino", d_values_trials, g_values_trials)

print(f"Direct time: {time_direct} vs Wino time: {time_wino}")

Direct time: (0.03495430946350098, array([0.21842409, 0.93066484, 1.13402111, 0.42391137])) vs Wino time: (60.94855332374573, Matrix([
[0.218424094535654],
[0.930664838785913],
[ 1.13402111338575],
[0.423911368403236]]))
